In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import io
import seaborn as sns
import glob
import pandas as pd
import polars as pl
import numba
from tqdm.notebook import tqdm


custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

cols_with_lists = [
        "nuc_starts",
        "nuc_lengths",
        "ref_nuc_starts",
        "ref_nuc_lengths",
        "msp_starts",
        "msp_lengths",
        "ref_msp_starts",
        "ref_msp_lengths",
        "m6a",
        "m6a_qual",
        "ref_m6a",
        "5mC",
        "ref_5mC",
    ]
    
import fibertools as ft

df = ft.read_fibertools_rs_all_file("../validation_data/m6ATP.ft.tbl.gz", n_rows=None, pandas=True)
df = df[df.m6a.isna() == False]


/mmfs1/gscratch/stergachislab/mvollger/miniconda3/envs/fiberseq-smk/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
df2 = ft.read_fibertools_rs_all_file("../validation_data/PS00243_ft.tbl.gz", n_rows=None, pandas=True)
df2 = df2[df2.m6a.isna() == False]

In [7]:
df2["RG"] = "PS00243_ML.fiberseq"
df = pd.concat([df, df2])

In [8]:
@numba.njit
def helper(m6a, seq, extend):
    kmers = []
    for bp in m6a:
        start = bp-extend
        end = bp+extend+1
        if start < 0 or end > len(seq):
            continue 
        kmers.append(seq[start:end])
    return kmers
    
def my_func(x, k=7):
    seq = x["fiber_sequence"]
    m6a = x["m6a"]
    extend = k//2
    return helper(m6a, seq, extend)

df["kmers"] = df.apply(my_func, axis=1)

In [168]:
kmer_counts = df[["RG", "kmers"]].explode("kmers").groupby(["RG", "kmers"], as_index=False).size()
kmer_counts

,RG,kmers,size
0,PS00231_ML.fiberseq,AAAAAAA,493
1,PS00231_ML.fiberseq,AAAAAAC,159
2,PS00231_ML.fiberseq,AAAAAAG,483
3,PS00231_ML.fiberseq,AAAAAAT,273
4,PS00231_ML.fiberseq,AAAAACA,457
...,...,...,...
57339,PS00237_ML.fiberseq,TTTTTGT,1488
57340,PS00237_ML.fiberseq,TTTTTTA,562
57341,PS00237_ML.fiberseq,TTTTTTC,864
57342,PS00237_ML.fiberseq,TTTTTTG,727


In [169]:
raw_kmer_counts = pd.read_csv("../validation_data/kmer_counts/kmers.tbl", sep=" ", names=["kmers", "raw_count", "RG"])
raw_kmer_counts

,kmers,raw_count,RG
0,AAAAAAA,725892,PS00231_ML.fiberseq
1,AAAAAAC,483392,PS00231_ML.fiberseq
2,AAAAAAG,726155,PS00231_ML.fiberseq
3,AAAAAAT,922676,PS00231_ML.fiberseq
4,AAAAACA,671901,PS00231_ML.fiberseq
...,...,...,...
114683,TTTTTGT,354390,PS00237_ML.fiberseq
114684,TTTTTTA,359404,PS00237_ML.fiberseq
114685,TTTTTTC,322372,PS00237_ML.fiberseq
114686,TTTTTTG,360760,PS00237_ML.fiberseq


In [170]:
print(kmer_counts.shape, raw_kmer_counts.shape)
out = kmer_counts.merge(raw_kmer_counts, on=["RG", "kmers"], how="right").replace(np.nan, 0)
out.to_csv("../Tables/m6a_kmers.tbl.gz", index=False)
out

(57344, 3) (114688, 3)


,RG,kmers,size,raw_count
0,PS00231_ML.fiberseq,AAAAAAA,493.0,725892
1,PS00231_ML.fiberseq,AAAAAAC,159.0,483392
2,PS00231_ML.fiberseq,AAAAAAG,483.0,726155
3,PS00231_ML.fiberseq,AAAAAAT,273.0,922676
4,PS00231_ML.fiberseq,AAAAACA,457.0,671901
...,...,...,...,...
114683,PS00237_ML.fiberseq,TTTTTGT,1488.0,354390
114684,PS00237_ML.fiberseq,TTTTTTA,562.0,359404
114685,PS00237_ML.fiberseq,TTTTTTC,864.0,322372
114686,PS00237_ML.fiberseq,TTTTTTG,727.0,360760
